# `FStream` Online Training

**[THIS IS WORK IN PROGRESS]**

This notebook performs online training of the **flow stream parent model** on the **car-shadow** sequence, so make sure you've run the [`FStream` Offline Training](fstream_offline_training.ipynb) notebook before running this one.


The online training of the `FStream` network is done by finetuning the parent model **on the first frame** of the video sequence. This is the only frame for which a mask is provided. It is augmented using scaling and vertical flipping. The network is trained for 500 iterations using the same training parameters as during offline training, except that deep supervision is disabled.

![](img/osvos_child.png)

To monitor training, run:
```
tensorboard --logdir E:\repos\tf-video-seg\tfvos\models\fstream_car-shadow
http://localhost:6006
```

In [ ]:
"""
fstream_online_training.ipynb

FStream online trainer

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Based on:
  - https://github.com/scaelles/OSVOS-TensorFlow/blob/master/osvos_parent_demo.py
    Written by Sergi Caelles (scaelles@vision.ee.ethz.ch)
    This file is part of the OSVOS paper presented in:
      Sergi Caelles, Kevis-Kokitsi Maninis, Jordi Pont-Tuset, Laura Leal-Taixe, Daniel Cremers, Luc Van Gool
      One-Shot Video Object Segmentation
      CVPR 2017
    Unknown code license
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from PIL import Image
import numpy as np
import tensorflow as tf
slim = tf.contrib.slim
import matplotlib.pyplot as plt

In [ ]:
# Import model files
import model
import datasets

## Configuration

In [ ]:
# Model paths
seq_name = "car-shadow"
segnet_stream = 'fstream'
parent_path = 'models/' + segnet_stream + '_parent/' + segnet_stream + '_parent-50000'
ckpt_name = segnet_stream + '_' + seq_name
logs_path = 'models/' + ckpt_name

# Online training parameters
gpu_id = 0
max_training_iters = 500
learning_rate = 1e-8
save_step = max_training_iters
side_supervision = 3
display_step = 10

## Dataset load

In [ ]:
# Load the DAVIS 2016 sequence
options = datasets._DEFAULT_DAVIS16_OPTIONS
options['use_cache'] = False
options['data_aug'] = True
# Set the following to wherever you have downloaded the DAVIS 2016 dataset
dataset_root = 'E:/datasets/davis2016/' if sys.platform.startswith("win") else '/media/EDrive/datasets/davis2016/'
test_frames = sorted(os.listdir(dataset_root + 'JPEGImages/480p/' + seq_name))
test_imgs = [dataset_root + 'JPEGImages/480p/' + seq_name + frame) for frame in test_frames]
train_imgs = [dataset_root + 'JPEGImages/480p/' + seq_name + '00000.jpg', 
              dataset_root + 'Annotations/480p' + seq_name + '00000.png']
dataset = datasets.davis16(train_imgs, test_imgs, dataset_root, options)

In [ ]:
# Display dataset configuration
dataset.print_config()

## Online Training

In [ ]:
# Finetune this branch of the binary segmentation network
with tf.Graph().as_default():
    with tf.device('/gpu:' + str(gpu_id)):
        global_step = tf.Variable(0, name='global_step', trainable=False)
        model.train_finetune(dataset, parent_path, side_supervision, learning_rate, logs_path, max_training_iters,
                             save_step, display_step, global_step, segnet_stream, iter_mean_grad=1, ckpt_name=ckpt_name)


## Testing

In [ ]:
# Result path (if you want to check how well this branch is doing on its own)
result_path = dataset_folder + 'Results/Segmentations/480p/' + ckpt_name

# Test this branch of the network
with tf.Graph().as_default():
    with tf.device('/gpu:' + str(gpu_id)):
        ckpt_path = logs_path + '/' + ckpt_name + '.ckpt-' + str(max_training_iters))
        model.test(dataset, ckpt_path, result_path)

## Visual evaluation

Let's load the original images and their predicted masks to get an idea of how this branch of the network is doing. Note that the first mask is displayed in red overlay, as it is given to us. The predicted masks are displayed using a green overlay.

In [ ]:
# Load results
frames = []
predicted_masks=[]
for test_frame in test_frames:
    frame_num = test_frame.split('.')[0]
    frame = np.array(Image.open(dataset_root + 'JPEGImages/480p/' + seq_name + '/' + test_frame))
    predicted_mask = np.array(Image.open(result_path + frame_num +'.png'))
    frames.append(frame)
    predicted_masks.append(predicted_mask)

In [ ]:
# Overlay the masks on top of the frames
frames_with_predictions = visualize.overlay_frames_with_predictions(frames, predicted_masks')

### Display individual frames

In [ ]:
visualize.display_images(frames_with_predictions)

### Display results as a video clip

In [ ]:
# Set path to video clips
video_clip_folder = dataset_root + 'clips/'
video_clip = video_clip_folder + ckpt_name + '.mp4'

# Combine images in a video clip
visualize.make_clip(video_clip, frames_with_predictions)

In [ ]:
# Display video
visualize.show_clip(video_clip)